# Pay Weekly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime, timedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

weekly total before running time.

In [ ]:
%%sql stat_accounts_completed  <<
SELECT count(*) FILTER (WHERE payment_method = 'PAD') AS PAD
,count(*) FILTER (WHERE payment_method = 'DRAWDOWN') AS BCOL
,count(*) FILTER (WHERE payment_method = 'ONLINE_BANKING') AS ONLINE_BANKING
,count(*) FILTER (WHERE payment_method = 'DIRECT_PAY') AS DIRECT_PAY
,count(*) FILTER (WHERE payment_method = 'CC') AS CC
,count(*) FILTER (WHERE payment_method = 'CASH') AS CASH
,count(*) FILTER (WHERE payment_method = 'CHEQUE') AS CHEQUE
,count(*) FILTER (WHERE payment_method = 'EJV') AS EJV
,count(*) FILTER (WHERE payment_method = 'WIRE') AS WIRE
,count(*) FILTER (WHERE payment_method = 'EFT') AS EFT
,count(*) FILTER (WHERE payment_method = 'INTERNAL') AS INTERNAL
FROM payment_accounts

Save to CSV

In [ ]:
filename = os.path.join(os.getcwd(), r'data/')+'weekly_pay_stats_till_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'

with open(filename, 'w') as f:      
    f.write('Total Number of Accounts on Each Payment Type - from payment_accounts table (overall):\n')
    stat_accounts_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_weekly_transactions_on_each_payment_completed  <<
SELECT count(*) FILTER (WHERE payment_method_code = 'PAD') AS PAD
,count(*) FILTER (WHERE payment_method_code = 'DRAWDOWN') AS BCOL
,count(*) FILTER (WHERE payment_method_code = 'ONLINE_BANKING') AS ONLINE_BANKING
,count(*) FILTER (WHERE payment_method_code = 'DIRECT_PAY') AS DIRECT_PAY
,count(*) FILTER (WHERE payment_method_code = 'CC') AS CC
,count(*) FILTER (WHERE payment_method_code = 'CASH') AS CASH
,count(*) FILTER (WHERE payment_method_code = 'CHEQUE') AS CHEQUE
,count(*) FILTER (WHERE payment_method_code = 'EJV') AS EJV
,count(*) FILTER (WHERE payment_method_code = 'WIRE') AS WIRE
,count(*) FILTER (WHERE payment_method_code = 'EFT') AS EFT
,count(*) FILTER (WHERE payment_method_code = 'INTERNAL') AS INTERNAL
FROM invoices
WHERE
invoice_status_code IN ('PAID', 'APPROVED')
AND date(created_on)  > date(current_date - 1 - interval '1 weeks')
AND date(created_on)  <= date(current_date - 1);

In [ ]:
with open(filename, 'a') as f:      
    f.write('\n\n Weekly Number of Transactions on Each Payment Type - from invoices table:\n')
    stat_weekly_transactions_on_each_payment_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_nro_weekly_completed  <<
SELECT count(*)
FROM invoices i
WHERE
i.corp_type_code = 'NRO'
AND i.payment_method_code IN ('PAD','DRAWDOWN','ONLINE_BANKING','DIRECT_PAY','CC','CASH','CHEQUE','EJV','WIRE','EFT','INTERNAL')
AND i.invoice_status_code IN ('PAID', 'APPROVED')
AND date(i.created_on at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '1 weeks')
AND date(i.created_on at time zone 'utc' at time zone 'pst')  <= date(current_date - 1);

In [ ]:
with open(filename, 'a') as f:      
    f.write('\n\n Weekly Number of NR Submitted in Premium Account - from invoices table:\n')
    stat_nro_weekly_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_nro_total_completed  <<
SELECT count(*)
FROM invoices i
WHERE
i.corp_type_code = 'NRO'
AND i.payment_method_code IN ('PAD','DRAWDOWN','ONLINE_BANKING','DIRECT_PAY','CC','CASH','CHEQUE','EJV','WIRE','EFT','INTERNAL')
AND i.invoice_status_code IN ('PAID', 'APPROVED');

In [ ]:
with open(filename, 'a') as f:      
    f.write('\n\n Total Number of NR Submitted in Premium Account - from invoices table:\n')
    stat_nro_total_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)